### MONITORING RULES

In [328]:
!pip install awswrangler

### EXCEL FILES WITH MAPEs OF TOP 15 PAYERS ARE UPLOADED

In [329]:
import awswrangler as wr
import boto3
import pandas as pd
from io import BytesIO
import numpy as np
from datetime import datetime, timedelta
from io import BytesIO # Para el WA
from io import StringIO
import joblib
import os

In [330]:
# S3 client
s3 = boto3.client('s3')

In [331]:
#Bucket and folder prefix
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
prefix_ABTv3_update = 'ABTv3_update/top_payers_abt_v3update/'

In [332]:
# I get list of folders in the bucket
def list_folders(bucket_name, prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')
    folders = [prefix['Prefix'].split('/')[-2] for prefix in response.get('CommonPrefixes', [])]
    return folders

In [333]:
# I call a function to list folders
folders_v3 = list_folders(bucket_name, prefix_ABTv3_update)

In [334]:
# I read the xlsx and consolidate

def read_files(bucket_name, prefix):
    dfs = []
    
    # Iterar sobre cada carpeta
    for folder_name in folders_v3:
        # Obtener la lista de objetos en la carpeta
        objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=f'{prefix}{folder_name}/')['Contents']
        
        # Buscar archivos xlsx en la carpeta
        excel_objects = [obj for obj in objects if obj['Key'].endswith('.xlsx') and '7d' not in obj['Key']]
        
        if excel_objects:
            print(excel_objects)
            # Leer el primer archivo xlsx encontrado
            obj = s3.get_object(Bucket=bucket_name, Key=excel_objects[0]['Key'])
            excel_data = obj['Body'].read() # No se puede leer con pandas directo de S3
            
            df = pd.read_excel(BytesIO(excel_data))
            # Agrego columna con el nombre de la carpeta
            df['folder_name'] = folder_name
            # Agregar el df a la lista
            dfs.append(df)
        else:
            print(f"No se encontraron archivos xlsx en la carpeta {folder_name}.")
    
    if dfs:
        # Concateno todo
        consolidated_df = pd.concat(dfs, ignore_index=True)
        return consolidated_df
    else:
        print("No se encontraron archivos xlsx en ninguna de las carpetas especificadas.")
        return None

In [335]:
# I call function to read xlsx and consolidate into one DF
df_top15 = read_files(bucket_name, prefix_ABTv3_update)

[{'Key': 'ABTv3_update/top_payers_abt_v3update/BANCO AGRICOLA_EL SALVADOR/predicciones_test_2d_BANCO AGRICOLA_EL SALVADOR.xlsx', 'LastModified': datetime.datetime(2024, 3, 25, 18, 45, 53, tzinfo=tzlocal()), 'ETag': '"9b406073e87517287809516d839f8a2d"', 'Size': 18494, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3_update/top_payers_abt_v3update/BANCO ATLANTIDA_HONDURAS/predicciones_test_2d_BANCO ATLANTIDA_HONDURAS.xlsx', 'LastModified': datetime.datetime(2024, 3, 25, 17, 14, 50, tzinfo=tzlocal()), 'ETag': '"efab2b35f556266e631250662095466d"', 'Size': 18913, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3_update/top_payers_abt_v3update/BANCO DE OCCIDENTE_HONDURAS/predicciones_test_2d_BANCO DE OCCIDENTE_HONDURAS.xlsx', 'LastModified': datetime.datetime(2024, 3, 25, 19, 18, 47, tzinfo=tzlocal()), 'ETag': '"6246eb42efc092e74e315d514e726e4c"', 'Size': 18856, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3_update/top_payers_abt_v3update/BANCO DE ORO (BDO)_PHILIPPINES/predicciones_test_2d_BANCO DE ORO

In [336]:
df_top15.folder_name.nunique()

15

In [337]:
df_top15.date.max()

Timestamp('2024-02-03 00:00:00')

### DATE OF ANALYSIS
### UP TO ONE DAY BEFORE THE FORECASTS THEY ALREADY HAVE (MAPE)##

In [338]:
df_top15=df_top15.loc[df_top15.date<'2024-01-28']

In [339]:
df_top15.tail(8)

,date,valor_real,valor_predicho,mape,error_abs,folder_name
3390,2024-01-20,418022.83,438828.622225,4.977190,20805.792225,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO
3391,2024-01-21,490154.49,541620.401018,10.499937,51465.911018,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO
3392,2024-01-22,476076.81,400419.571516,15.891813,75657.238484,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO
3393,2024-01-23,468738.81,452307.372134,3.505457,16431.437866,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO
3394,2024-01-24,366077.85,397733.669243,8.647292,31655.819243,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO
3395,2024-01-25,402488.65,421819.511674,4.802834,19330.861674,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO
3396,2024-01-26,439061.39,441922.341655,0.651606,2860.951655,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO
3397,2024-01-27,423683.10,462898.462382,9.255824,39215.362382,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO


In [340]:
df_top15.folder_name.unique()

array(['BANCO AGRICOLA_EL SALVADOR', 'BANCO ATLANTIDA_HONDURAS',
       'BANCO DE OCCIDENTE_HONDURAS', 'BANCO DE ORO (BDO)_PHILIPPINES',
       'BANCO INDUSTRIAL_GUATEMALA', 'BANCO RENDIMENTO_BRAZIL',
       'BANCOPPEL (APPRIZA)_MEXICO', 'BANORTE (UT)_MEXICO',
       'BANRURAL (RYT)_GUATEMALA', 'BBVA - BANCOMER (BTS)_MEXICO',
       'CARIBE EXPRESS_DOMINICAN REPUBLIC', 'ELEKTRA (MEXICO)_MEXICO',
       'ENVIOS CON CLABE (APPRIZA)_MEXICO', 'G Y T CONTINENTAL_GUATEMALA',
       'TELECOMM TELEGRAFOS (APPRIZA)_MEXICO'], dtype=object)

In [341]:
lista_top15=['BANCO AGRICOLA_EL SALVADOR', 'BANCO ATLANTIDA_HONDURAS',
       'BANCO DE OCCIDENTE_HONDURAS', 'BANCO DE ORO (BDO)_PHILIPPINES',
       'BANCO INDUSTRIAL_GUATEMALA', 'BANCO RENDIMENTO_BRAZIL',
       'BANCOPPEL (APPRIZA)_MEXICO', 'BANORTE (UT)_MEXICO',
       'BANRURAL (RYT)_GUATEMALA', 'BBVA - BANCOMER (BTS)_MEXICO',
       'CARIBE EXPRESS_DOMINICAN REPUBLIC', 'ELEKTRA (MEXICO)_MEXICO',
       'ENVIOS CON CLABE (APPRIZA)_MEXICO', 'G Y T CONTINENTAL_GUATEMALA',
       'TELECOMM TELEGRAFOS (APPRIZA)_MEXICO']

## LOAD THE RECENT INFERENCES AND THEIR MAPEs

In [342]:
# Set up S3 client
client = boto3.client('s3')
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
path_MAPE = 'ABTv3_update/MAPE_inferences_2d_rules_monitoring/' # Folder under analysis

In [343]:
# Specify the CSV file key
csv_key = None

# List objects in the S3 path
response = client.list_objects(Bucket=bucket_name, Prefix=path_MAPE)

# Find the CSV file in the S3 path
for obj in response.get('Contents', []):
    if obj['Key'].endswith('.csv'):
        csv_key = obj['Key']
        break
        
# Check if CSV file is found
if csv_key is not None:
    # Read CSV content from S3
    csv_response = client.get_object(Bucket=bucket_name, Key=csv_key)
    csv_content = csv_response['Body'].read().decode('utf-8')

    # Transform CSV content to DataFrame
    data = pd.read_csv(StringIO(csv_content))
    data['pred_date'] = pd.to_datetime(data['pred_date']).dt.date
    print("CSV file loaded")
else:
    print("No CSV file found in the specified S3 path.")

CSV file loaded


In [344]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   processing_date  204 non-null    object 
 1   pred_date        204 non-null    object 
 2   pred             204 non-null    float64
 3   payer_country    204 non-null    object 
 4   id_main_branch   204 non-null    object 
 5   id_country       204 non-null    object 
 6   amount           204 non-null    float64
 7   abs_error        204 non-null    float64
 8   MAPE             204 non-null    float64
dtypes: float64(4), object(5)
memory usage: 14.5+ KB


In [345]:
data['pred_date'] = pd.to_datetime(data['pred_date'])

In [346]:
df_top15['date'] = pd.to_datetime(df_top15['date'])

In [347]:
##NEGATIVE PREDICTED VALUES ARE REPLACED
valor_a_reemplazar = 0
df_top15['valor_predicho'] = df_top15['valor_predicho'].apply(lambda x: valor_a_reemplazar if x < 0 else x)
data['pred'] = data['pred'].apply(lambda x: valor_a_reemplazar if x < 0 else x)

In [348]:
###HOMOGENIZE VARIABLES##

In [349]:
df_top15 = df_top15.rename(columns={'valor_predicho': 'pred', 'valor_real': 'amount', 'date': 'pred_date', 'folder_name': 'payer_country' })

In [350]:
data = data.rename(columns={'MAPE': 'mape','abs_error': 'error_abs'})

In [351]:
df_top15.head(2)

,pred_date,amount,pred,mape,error_abs,payer_country
0,2023-06-22,333877.5,383497.896918,14.861857,49620.396918,BANCO AGRICOLA_EL SALVADOR
1,2023-06-23,472255.0,526814.668818,11.553010,54559.668818,BANCO AGRICOLA_EL SALVADOR


In [352]:
data=data.loc[:,['pred_date', 'amount', 'pred', 'mape', 'error_abs', 'payer_country']]
data.head(2)

,pred_date,amount,pred,mape,error_abs,payer_country
0,2024-01-28,42919.0,43509.565347,1.376000,590.565347,24XORO_MEXICO
1,2024-01-29,37422.0,51645.303146,38.007865,14223.303146,24XORO_MEXICO


In [354]:
##ONLY THE FIRST 15 PAYERS ARE KEPT##

In [355]:
data = data[data['payer_country'].isin(lista_top15)]

In [356]:
###WE MERGE THE DF OF "NEW" PREDICTIONS (In data)  WITH THE HISTORY MAPEs FOR THE TOP 15 PAYERS##

In [357]:
data.payer_country.unique()

array(['BANCO AGRICOLA_EL SALVADOR', 'BANCO ATLANTIDA_HONDURAS',
       'BANCO DE OCCIDENTE_HONDURAS', 'BANCO DE ORO (BDO)_PHILIPPINES',
       'BANCO INDUSTRIAL_GUATEMALA', 'BANCO RENDIMENTO_BRAZIL',
       'BANCOPPEL (APPRIZA)_MEXICO', 'BANORTE (UT)_MEXICO',
       'BANRURAL (RYT)_GUATEMALA', 'BBVA - BANCOMER (BTS)_MEXICO',
       'CARIBE EXPRESS_DOMINICAN REPUBLIC', 'ELEKTRA (MEXICO)_MEXICO',
       'ENVIOS CON CLABE (APPRIZA)_MEXICO', 'G Y T CONTINENTAL_GUATEMALA',
       'TELECOMM TELEGRAFOS (APPRIZA)_MEXICO'], dtype=object)

In [358]:
df_top15.payer_country.unique()

array(['BANCO AGRICOLA_EL SALVADOR', 'BANCO ATLANTIDA_HONDURAS',
       'BANCO DE OCCIDENTE_HONDURAS', 'BANCO DE ORO (BDO)_PHILIPPINES',
       'BANCO INDUSTRIAL_GUATEMALA', 'BANCO RENDIMENTO_BRAZIL',
       'BANCOPPEL (APPRIZA)_MEXICO', 'BANORTE (UT)_MEXICO',
       'BANRURAL (RYT)_GUATEMALA', 'BBVA - BANCOMER (BTS)_MEXICO',
       'CARIBE EXPRESS_DOMINICAN REPUBLIC', 'ELEKTRA (MEXICO)_MEXICO',
       'ENVIOS CON CLABE (APPRIZA)_MEXICO', 'G Y T CONTINENTAL_GUATEMALA',
       'TELECOMM TELEGRAFOS (APPRIZA)_MEXICO'], dtype=object)

In [359]:
df_top15 = df_top15.sort_values(by=['payer_country', 'pred_date'])

In [360]:
df_top15 = pd.concat([df_top15, data])

In [361]:
df_top15.pred_date.max()

Timestamp('2024-01-29 00:00:00')

In [362]:
df_top15.head()

,pred_date,amount,pred,mape,error_abs,payer_country
0,2023-06-22,333877.5,383497.896918,14.861857,49620.396918,BANCO AGRICOLA_EL SALVADOR
1,2023-06-23,472255.0,526814.668818,11.553010,54559.668818,BANCO AGRICOLA_EL SALVADOR
2,2023-06-24,471284.0,489445.019909,3.853519,18161.019909,BANCO AGRICOLA_EL SALVADOR
3,2023-06-25,387389.0,409110.539878,5.607165,21721.539878,BANCO AGRICOLA_EL SALVADOR
4,2023-06-26,398244.0,409621.536758,2.856926,11377.536758,BANCO AGRICOLA_EL SALVADOR


In [363]:
df_top15.pred_date.max()

Timestamp('2024-01-29 00:00:00')

In [364]:
df_top15

,pred_date,amount,pred,mape,error_abs,payer_country
0,2023-06-22,333877.50,3.834979e+05,14.861857,49620.396918,BANCO AGRICOLA_EL SALVADOR
1,2023-06-23,472255.00,5.268147e+05,11.553010,54559.668818,BANCO AGRICOLA_EL SALVADOR
2,2023-06-24,471284.00,4.894450e+05,3.853519,18161.019909,BANCO AGRICOLA_EL SALVADOR
3,2023-06-25,387389.00,4.091105e+05,5.607165,21721.539878,BANCO AGRICOLA_EL SALVADOR
4,2023-06-26,398244.00,4.096215e+05,2.856926,11377.536758,BANCO AGRICOLA_EL SALVADOR
...,...,...,...,...,...,...
112,2024-01-29,1047698.27,1.048350e+06,0.062160,651.250487,ENVIOS CON CLABE (APPRIZA)_MEXICO
122,2024-01-28,480077.61,4.877841e+05,1.605253,7706.460042,G Y T CONTINENTAL_GUATEMALA
123,2024-01-29,530834.20,5.398347e+05,1.695539,9000.498242,G Y T CONTINENTAL_GUATEMALA
178,2024-01-28,527858.74,5.092807e+05,3.519510,18578.039540,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO


In [365]:
df_top15 = df_top15.sort_values(by=['payer_country', 'pred_date'])

In [366]:
##WE OBTAIN THE MOVING AVERAGE OF THE MAPE OF THE PREVIOUS 7 DAYS.
df_top15['mape_7_days_avg'] = df_top15.groupby('payer_country')['mape'].rolling(window=7, min_periods=5).mean().reset_index(level=0, drop=True).shift(1)

In [367]:
df_top15.pred_date.max()

Timestamp('2024-01-29 00:00:00')

### DEFINITION OF RULES

### CONDITION 1: THAT THE 7-DAY MOVING AVERAGE OF THE LAST TWO MONDAYS GROWS:

In [368]:
##MAPE AVERAGE IN TEST
##REFERENCE HISTORICAL MAPEs ARE CALCULATED BETWEEN 01/01/2021 AND 12/18/23
mape_mean_period = df_top15[(df_top15['pred_date'] >= '2023-06-22') & (df_top15['pred_date'] <= '2023-12-18')].groupby('payer_country')['mape'].mean()

# FUNCTION TO CALCULATE THE MOVING AVERAGE OF THE LAST 7 DAYS EXCLUDING VALUES 10 TIMES OR MORE HIGHER TAHN THE AVERAGE MAPE.
def mape_7_days_avg(group):
    
    high_mape_values = group[group['mape'] >= 10 * mape_mean_period[group.name]]
    group.loc[high_mape_values.index, 'mape'] = None  # Replace high values with None

    #Calculate the moving average of the last 7 days for the reference period (06/2023 - 12-2023)
    group['mape_7_days_avg'] = group.groupby('payer_country')['mape'].rolling(window=7, min_periods=5).mean().reset_index(level=0, drop=True).shift(1)

    return group

# Apply the mape_7_days_avg function to each group

data_processed = df_top15.groupby('payer_country').apply(mape_7_days_avg)

# Reset index
data_processed.reset_index(drop=True, inplace=True)

#Apply the grouping function and obtain the variable CONDITION 1 for each 'folder_name'.

def check_growth(group):
    ###DETECT THE LAST MONDAY IN THE SERIES
    last_monday = group[group['pred_date'].dt.dayofweek == 0].max()['pred_date']
    ### EXTRAMING THE MAPEs MOBILE AVERAGE FOR THE 7 DAYS PRIOR TO THAT MONDAY
    mape_last_monday = group[group['pred_date'] == last_monday]['mape_7_days_avg'].iloc[0]
    ###DETECT THE PREVIOUS MONDAY
    previous_monday = last_monday - pd.DateOffset(7)
    ###OBTAINING THE MOVING AVERAGE OF MAPS FOR THE PREVIOUS SEVEN DAYS
    mape_previous_monday = group[group['pred_date'] == previous_monday]['mape_7_days_avg'].iloc[0]
    ###DETECT THE MONDAY BEFORE THE SECOND MONDAY
    previous_previous_monday = previous_monday - pd.DateOffset(7)
     ###OBTAINING THE MOVING AVERAGE OF MAPES FOR THE PREVIOUS SEVEN DAYS
    mape_previous_previous_monday = group[group['pred_date'] == previous_previous_monday]['mape_7_days_avg'].iloc[0]

    # CREATE A DATAFRAME WITH ADDITIONAL COLUMNS
    result = pd.DataFrame({
        'last_check_date': [last_monday],
        'payer_country': [group['payer_country'].iloc[0]], # El nombre del folder_name es el mismo para todos los registros del grupo
        'mape_last_monday': [mape_last_monday],
        'mape_previous_monday': [mape_previous_monday],
        'mape_previous_previous_monday': [mape_previous_previous_monday]
    })

    # ADD CONDITION 1 COLUMN
    if mape_last_monday > mape_previous_monday and mape_previous_monday > mape_previous_previous_monday:
        result['CONDITION1'] = True
    else:
        result['CONDITION1'] = False

    return result

# APPLY THE CHECK_GROWTH FUNCTION AND CONCATENATE RESULTS
result = pd.concat([check_growth(group) for _, group in data_processed.groupby('payer_country')], ignore_index=True)

print(result)

   last_check_date                         payer_country  mape_last_monday  \
0       2024-01-29            BANCO AGRICOLA_EL SALVADOR          5.896972   
1       2024-01-29              BANCO ATLANTIDA_HONDURAS          4.357833   
2       2024-01-29           BANCO DE OCCIDENTE_HONDURAS          9.096413   
3       2024-01-29        BANCO DE ORO (BDO)_PHILIPPINES         15.381394   
4       2024-01-29            BANCO INDUSTRIAL_GUATEMALA          5.201585   
5       2024-01-29               BANCO RENDIMENTO_BRAZIL         30.537591   
6       2024-01-29            BANCOPPEL (APPRIZA)_MEXICO          5.595922   
7       2024-01-29                   BANORTE (UT)_MEXICO          5.955230   
8       2024-01-29              BANRURAL (RYT)_GUATEMALA          9.843966   
9       2024-01-29          BBVA - BANCOMER (BTS)_MEXICO          5.332800   
10      2024-01-29     CARIBE EXPRESS_DOMINICAN REPUBLIC          6.533318   
11      2024-01-29               ELEKTRA (MEXICO)_MEXICO        

/tmp/ipykernel_21/3157160575.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_processed = df_top15.groupby('payer_country').apply(mape_7_days_avg)


In [369]:
mape_mean_period2=data_processed[(data_processed['pred_date'] >= '2023-06-22') & (data_processed['pred_date'] <= '2023-12-18')].groupby('payer_country')['mape'].mean()
##AVERAGE MAPE OF THE TEST PERIOD (2023-06 2023-12) WITHOUT MAPEs > 10X TEST PERIOD
mape_mean_period2

payer_country
BANCO AGRICOLA_EL SALVADOR               7.094948
BANCO ATLANTIDA_HONDURAS                 8.089944
BANCO DE OCCIDENTE_HONDURAS              7.828954
BANCO DE ORO (BDO)_PHILIPPINES          15.246002
BANCO INDUSTRIAL_GUATEMALA               7.838171
BANCO RENDIMENTO_BRAZIL                 18.856583
BANCOPPEL (APPRIZA)_MEXICO               8.023823
BANORTE (UT)_MEXICO                      9.350056
BANRURAL (RYT)_GUATEMALA                 8.651675
BBVA - BANCOMER (BTS)_MEXICO             9.952609
CARIBE EXPRESS_DOMINICAN REPUBLIC        8.666772
ELEKTRA (MEXICO)_MEXICO                  6.994206
ENVIOS CON CLABE (APPRIZA)_MEXICO        8.032867
G Y T CONTINENTAL_GUATEMALA              9.788683
TELECOMM TELEGRAFOS (APPRIZA)_MEXICO    11.625459
Name: mape, dtype: float64

### CONDITION2: THE MAPE OF THE LAST WEEK IS NOT GREATER THAN THE MAXIMUM 7-DAY MOVING AVERAGE HISTORICAL MAXIMUM OF THE TEST PERIOD

In [370]:
mape_max_period=data_processed[(data_processed['pred_date'] >= '2023-06-22') & (data_processed['pred_date'] <= '2023-12-18')].groupby('payer_country')['mape_7_days_avg'].max()
mape_max_period

payer_country
BANCO AGRICOLA_EL SALVADOR              13.488821
BANCO ATLANTIDA_HONDURAS                16.010195
BANCO DE OCCIDENTE_HONDURAS             17.638946
BANCO DE ORO (BDO)_PHILIPPINES          28.174718
BANCO INDUSTRIAL_GUATEMALA              24.976621
BANCO RENDIMENTO_BRAZIL                 42.138393
BANCOPPEL (APPRIZA)_MEXICO              21.307336
BANORTE (UT)_MEXICO                     17.853906
BANRURAL (RYT)_GUATEMALA                18.573531
BBVA - BANCOMER (BTS)_MEXICO            18.435182
CARIBE EXPRESS_DOMINICAN REPUBLIC       20.647582
ELEKTRA (MEXICO)_MEXICO                 18.153615
ENVIOS CON CLABE (APPRIZA)_MEXICO       19.877006
G Y T CONTINENTAL_GUATEMALA             23.019552
TELECOMM TELEGRAFOS (APPRIZA)_MEXICO    24.395719
Name: mape_7_days_avg, dtype: float64

In [371]:
###WE ADD THE MAXIMUM OF THE 7-DAY MOVING AVERAGE OF THE MAPEs IN THE TEST PERIOD
df_monitoring=pd.merge(result, mape_max_period, on='payer_country', how='outer')

In [372]:
df_monitoring

,last_check_date,payer_country,mape_last_monday,mape_previous_monday,mape_previous_previous_monday,CONDITION1,mape_7_days_avg
0,2024-01-29,BANCO AGRICOLA_EL SALVADOR,5.896972,11.808818,9.431614,False,13.488821
1,2024-01-29,BANCO ATLANTIDA_HONDURAS,4.357833,5.389756,10.746015,False,16.010195
2,2024-01-29,BANCO DE OCCIDENTE_HONDURAS,9.096413,11.656978,13.328649,False,17.638946
3,2024-01-29,BANCO DE ORO (BDO)_PHILIPPINES,15.381394,14.833366,16.661164,False,28.174718
4,2024-01-29,BANCO INDUSTRIAL_GUATEMALA,5.201585,7.610178,9.756218,False,24.976621
5,2024-01-29,BANCO RENDIMENTO_BRAZIL,30.537591,25.152328,19.417284,True,42.138393
6,2024-01-29,BANCOPPEL (APPRIZA)_MEXICO,5.595922,8.668374,13.626734,False,21.307336
7,2024-01-29,BANORTE (UT)_MEXICO,5.955230,10.718542,10.743145,False,17.853906
8,2024-01-29,BANRURAL (RYT)_GUATEMALA,9.843966,14.021984,8.786395,False,18.573531
9,2024-01-29,BBVA - BANCOMER (BTS)_MEXICO,5.332800,11.143071,16.143505,False,18.435182


In [373]:
df_monitoring.rename(columns={'mape_7_days_avg': 'mape_7_days_avg_mobile_max'}, inplace=True)

In [374]:
###GENERATE CONDITION 2 AND CONDITION 3
def generate_condition2(row):
    if  row['mape_last_monday']  > row['mape_7_days_avg_mobile_max'] * 1.05:
        return True
    else:
        return False

def generate_condition3(row):
    if  row['mape_last_monday']  >  row['mape_7_days_avg_mobile_max'] * 1.10:
        return True
    else:
        return False

# Aplicar la función a lo largo de las filas del DataFrame
df_monitoring['CONDITION2'] = df_monitoring.apply(generate_condition2, axis=1)
df_monitoring['CONDITION3'] = df_monitoring.apply(generate_condition3, axis=1)

In [375]:
df_monitoring

,last_check_date,payer_country,mape_last_monday,mape_previous_monday,mape_previous_previous_monday,CONDITION1,mape_7_days_avg_mobile_max,CONDITION2,CONDITION3
0,2024-01-29,BANCO AGRICOLA_EL SALVADOR,5.896972,11.808818,9.431614,False,13.488821,False,False
1,2024-01-29,BANCO ATLANTIDA_HONDURAS,4.357833,5.389756,10.746015,False,16.010195,False,False
2,2024-01-29,BANCO DE OCCIDENTE_HONDURAS,9.096413,11.656978,13.328649,False,17.638946,False,False
3,2024-01-29,BANCO DE ORO (BDO)_PHILIPPINES,15.381394,14.833366,16.661164,False,28.174718,False,False
4,2024-01-29,BANCO INDUSTRIAL_GUATEMALA,5.201585,7.610178,9.756218,False,24.976621,False,False
5,2024-01-29,BANCO RENDIMENTO_BRAZIL,30.537591,25.152328,19.417284,True,42.138393,False,False
6,2024-01-29,BANCOPPEL (APPRIZA)_MEXICO,5.595922,8.668374,13.626734,False,21.307336,False,False
7,2024-01-29,BANORTE (UT)_MEXICO,5.955230,10.718542,10.743145,False,17.853906,False,False
8,2024-01-29,BANRURAL (RYT)_GUATEMALA,9.843966,14.021984,8.786395,False,18.573531,False,False
9,2024-01-29,BBVA - BANCOMER (BTS)_MEXICO,5.332800,11.143071,16.143505,False,18.435182,False,False


### RECOMMENDED ACTIONS

In [376]:
###RECOMMENDED ACTIONS ARE GENERATED

def actions_recommended(row):
    if  row['CONDITION1'] == True and row['CONDITION2']==True and row['CONDITION3']==False:
        return 'REQUIRES CALIBRATION'
    elif row['CONDITION1'] == True and row['CONDITION3']==True:
        return 'REQUIRES RE-TRAINING'
    else:
        return 'KEEP CURRENT MODEL'

df_monitoring['RECOMMENDED_ACTIONS'] = df_monitoring.apply(actions_recommended, axis=1)

In [377]:
df_monitoring

,last_check_date,payer_country,mape_last_monday,mape_previous_monday,mape_previous_previous_monday,CONDITION1,mape_7_days_avg_mobile_max,CONDITION2,CONDITION3,RECOMMENDED_ACTIONS
0,2024-01-29,BANCO AGRICOLA_EL SALVADOR,5.896972,11.808818,9.431614,False,13.488821,False,False,KEEP CURRENT MODEL
1,2024-01-29,BANCO ATLANTIDA_HONDURAS,4.357833,5.389756,10.746015,False,16.010195,False,False,KEEP CURRENT MODEL
2,2024-01-29,BANCO DE OCCIDENTE_HONDURAS,9.096413,11.656978,13.328649,False,17.638946,False,False,KEEP CURRENT MODEL
3,2024-01-29,BANCO DE ORO (BDO)_PHILIPPINES,15.381394,14.833366,16.661164,False,28.174718,False,False,KEEP CURRENT MODEL
4,2024-01-29,BANCO INDUSTRIAL_GUATEMALA,5.201585,7.610178,9.756218,False,24.976621,False,False,KEEP CURRENT MODEL
5,2024-01-29,BANCO RENDIMENTO_BRAZIL,30.537591,25.152328,19.417284,True,42.138393,False,False,KEEP CURRENT MODEL
6,2024-01-29,BANCOPPEL (APPRIZA)_MEXICO,5.595922,8.668374,13.626734,False,21.307336,False,False,KEEP CURRENT MODEL
7,2024-01-29,BANORTE (UT)_MEXICO,5.955230,10.718542,10.743145,False,17.853906,False,False,KEEP CURRENT MODEL
8,2024-01-29,BANRURAL (RYT)_GUATEMALA,9.843966,14.021984,8.786395,False,18.573531,False,False,KEEP CURRENT MODEL
9,2024-01-29,BBVA - BANCOMER (BTS)_MEXICO,5.332800,11.143071,16.143505,False,18.435182,False,False,KEEP CURRENT MODEL


### THE MONITORING RESULTS DATASET IS STORED

In [ ]:
# Nombre del archivo con la fecha
df_monitoring['last_check_date'] = df_monitoring['last_check_date'].dt.strftime('%Y%m%d')
file_name = f"df_monitoring_{df_monitoring['last_check_date'].iloc[0]}.csv"

# Dejo el archivo en S3
bucket = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
prefix_abt = 'MONITORING_RULES'

# Ruta S3
s3_path = f"s3://{bucket}/{prefix_abt}/{file_name}"

# Guardar el DataFrame en CSV en S3
wr.s3.to_csv(df_monitoring, path=s3_path, index=False)